In [1]:
# to access functions from root directory
import sys
sys.path.append('/data/ad181/RemoteDir/multilevel_ppo')

In [2]:
%matplotlib notebook
import numpy as np
import time
import pickle
import os
import matplotlib.pyplot as plt
from copy import copy, deepcopy
from tqdm.notebook import trange, tqdm

import gym
from stable_baselines3.ppo import PPO, MlpPolicy
from stable_baselines3.ppo_multi_level import PPO_ML
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env.subproc_vec_multi_level_env import SubprocVecMultiLevelEnv
from stable_baselines3.common.envs.multi_level_ressim_env import MultiLevelRessimEnv
from stable_baselines3.common.logger import configure

from utils.custom_eval_callback import CustomEvalCallback, CustomEvalCallbackParallel
from utils.plot_functions import plot_learning
from utils.env_evaluate_functions import eval_actions

In [3]:
seed=1
case='ppo_1l'
data_dir='./data'
log_dir='./data/'+case

In [4]:
os.makedirs(data_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

In [5]:
with open('../envs_params/env_data_v1/env_train_dict.pkl', 'rb') as input:
    env_ck_dict = pickle.load(input)

In [6]:
# generate dictionaries for env (env_dict_), n_steps (T_ml) and minibatch (M_ml) for `n_level`s
n_levels=1
fine_level = len(env_ck_dict)
env_dict_ = {}
for i,l in enumerate(range(fine_level-n_levels, fine_level)):
    print(i+1,'->',l+1)
    env_dict_[i+1] = env_ck_dict[l+1]

1 -> 4


In [ ]:
for seed in range(1,4):
    if True: 
        print(f'seed {seed}')
        log_dir = './data/'+case+'/seed_'+str(seed)
        os.makedirs(log_dir, exist_ok=True)
        T = 100 # n_steps
        N = 50 # number of actors
        M = 500 # minibatch size
        I = 1200 # number of iterations
        K = 20 # number of epochs
        
        log_interval = I/120
        
        fine_level = len(env_dict_)
        
        print('generate callback ...')
        eval_callback = CustomEvalCallback( env_dict_[fine_level], 
                                            best_model_save_path=str(log_dir)+'/best_model', 
                                            n_eval_episodes=1,
                                            log_path=str(log_dir)+'/results_eval', 
                                            eval_freq=log_interval*T)
        
        print('vectorize environment ...')
        
        # generate PPO_ML parameters for MLMC analysis. 
        # we choose same n_steps and batch_size values on levels because only fine level values are used in the analysis
        env_dict = {}
        n_steps_dict = {}
        batch_size_dict = {}
        for env, level in zip(env_dict_.values(), env_dict_.keys()):
            print(f"vectorize env level {level}")
            env_dict[level] = make_vec_env( MultiLevelRessimEnv, 
                                    n_envs=N, 
                                    seed=seed, 
                                    env_kwargs= {"ressim_params":env.ressim_params, "level":env.level}, 
                                    vec_env_cls=SubprocVecMultiLevelEnv )
            n_steps_dict[level] = T
            batch_size_dict[level] = M
        
        print(env_dict_[level].observation_space)
        print('model definition ..')
        model = PPO_ML(policy=MlpPolicy,
                           env=env_dict,
                           learning_rate = 1e-6,
                           n_steps = n_steps_dict,
                           batch_size = batch_size_dict,
                           n_epochs = K,
                           clip_range = 0.1,
                           ent_coef = 0.001,
                           vf_coef = 0.5,
                           policy_kwargs = dict(net_arch=[70,70,50], log_std_init=-2.9),
                           verbose = 1,
                           seed = seed,
                           target_kl = 0.05,
                           device = "auto")
        # set logger for the model
        new_logger = configure(log_dir)
        model.set_logger(new_logger)
        print('policy learning ..')
        model.learn(total_timesteps=N*T*I, callback=eval_callback)
        model.save(log_dir+'/PPO', exclude=['env_dict'])
        del model
        for level in env_dict.keys():
            env_dict[level].close()


seed 1
generate callback ...
vectorize environment ...
vectorize env level 1
Box(-1.0, 1.0, (35,), float64)
model definition ..
Using cuda device
Logging to ./data/ppo_1l/seed_1
policy learning ..


/data/ad181/RemoteDir/multilevel_ppo/utils/custom_eval_callback.py:146: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.subproc_vec_multi_level_env.SubprocVecMultiLevelEnv object at 0x7f74335eed90> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7f74335e6ad0>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 5        |
|    ep_rew_mean     | 0.679    |
| time/              |          |
|    fps             | 56       |
|    iterations      | 1        |
|    time_elapsed    | 88       |
|    total_timesteps | 5000     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5           |
|    ep_rew_mean          | 0.683       |
| time/                   |             |
|    fps                  | 61          |
|    iterations           | 2           |
|    time_elapsed         | 163         |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.001055611 |
|    clip_fraction        | 0.0089      |
|    clip_range           | 0.1         |
|    entropy_loss         | -31.1       |
|    explained_variance   | -7.87       |
|    learning_rate        | 1e

/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):


Eval num_timesteps=50000, episode_reward=0.69 +/- 0.00
Episode length: 5.00 +/- 0.00
New best mean reward!
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 5            |
|    mean_reward          | 0.686        |
| rollout/                |              |
|    ep_len_mean          | 5            |
|    ep_rew_mean          | 0.678        |
| time/                   |              |
|    fps                  | 64           |
|    iterations           | 10           |
|    time_elapsed         | 778          |
|    total_timesteps      | 50000        |
| train/                  |              |
|    approx_kl            | 0.0010675994 |
|    clip_fraction        | 0.00679      |
|    clip_range           | 0.1          |
|    entropy_loss         | -31.1        |
|    explained_variance   | -3.92        |
|    learning_rate        | 1e-06        |
|    loss                 | 0.125        |
|    n_updates            | 180  

...

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5            |
|    ep_rew_mean          | 0.846        |
| time/                   |              |
|    fps                  | 68           |
|    iterations           | 875          |
|    time_elapsed         | 64264        |
|    total_timesteps      | 4375000      |
| train/                  |              |
|    approx_kl            | 0.0013849032 |
|    clip_fraction        | 0.03         |
|    clip_range           | 0.1          |
|    entropy_loss         | -31.4        |
|    explained_variance   | 0.977        |
|    learning_rate        | 1e-06        |
|    loss                 | 0.03         |
|    n_updates            | 17480        |
|    policy_gradient_loss | -0.00286     |
|    std                  | 0.0544       |
|    value_loss           | 0.00154      |
------------------------------------------
------------------------------------------
| rollout/ 

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5            |
|    ep_rew_mean          | 0.847        |
| time/                   |              |
|    fps                  | 68           |
|    iterations           | 884          |
|    time_elapsed         | 64928        |
|    total_timesteps      | 4420000      |
| train/                  |              |
|    approx_kl            | 0.0014875763 |
|    clip_fraction        | 0.0468       |
|    clip_range           | 0.1          |
|    entropy_loss         | -31.4        |
|    explained_variance   | 0.977        |
|    learning_rate        | 1e-06        |
|    loss                 | 0.0259       |
|    n_updates            | 17660        |
|    policy_gradient_loss | -0.00385     |
|    std                  | 0.0543       |
|    value_loss           | 0.00154      |
------------------------------------------
----------------------------------------
| rollout/   

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5            |
|    ep_rew_mean          | 0.849        |
| time/                   |              |
|    fps                  | 68           |
|    iterations           | 893          |
|    time_elapsed         | 65594        |
|    total_timesteps      | 4465000      |
| train/                  |              |
|    approx_kl            | 0.0017251879 |
|    clip_fraction        | 0.0524       |
|    clip_range           | 0.1          |
|    entropy_loss         | -31.4        |
|    explained_variance   | 0.977        |
|    learning_rate        | 1e-06        |
|    loss                 | 0.0269       |
|    n_updates            | 17840        |
|    policy_gradient_loss | -0.00372     |
|    std                  | 0.0543       |
|    value_loss           | 0.00155      |
------------------------------------------
-----------------------------------------
| rollout/  

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5            |
|    ep_rew_mean          | 0.848        |
| time/                   |              |
|    fps                  | 68           |
|    iterations           | 902          |
|    time_elapsed         | 66262        |
|    total_timesteps      | 4510000      |
| train/                  |              |
|    approx_kl            | 0.0012831014 |
|    clip_fraction        | 0.0355       |
|    clip_range           | 0.1          |
|    entropy_loss         | -31.4        |
|    explained_variance   | 0.976        |
|    learning_rate        | 1e-06        |
|    loss                 | 0.0275       |
|    n_updates            | 18020        |
|    policy_gradient_loss | -0.00356     |
|    std                  | 0.0543       |
|    value_loss           | 0.0016       |
------------------------------------------
------------------------------------------
| rollout/ 

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5            |
|    ep_rew_mean          | 0.85         |
| time/                   |              |
|    fps                  | 68           |
|    iterations           | 911          |
|    time_elapsed         | 66928        |
|    total_timesteps      | 4555000      |
| train/                  |              |
|    approx_kl            | 0.0016817048 |
|    clip_fraction        | 0.0393       |
|    clip_range           | 0.1          |
|    entropy_loss         | -31.4        |
|    explained_variance   | 0.978        |
|    learning_rate        | 1e-06        |
|    loss                 | 0.0241       |
|    n_updates            | 18200        |
|    policy_gradient_loss | -0.00357     |
|    std                  | 0.0543       |
|    value_loss           | 0.00146      |
------------------------------------------
------------------------------------------
| rollout/ 